## This is the only cell you need to change ##
### Possible file formats ###
- CSV
- excel

### Possible output formats ###
- long format where each line contains one value
- wide table-style format where values for different years (and in case of trade parameters regions) are grouped per line

### Possible processing options ###
- both creates the dataset for parameters and timeseries
- parameters_only for only regular parameters (saves time since timeseries, especially with excel, takes a lot of time)
- timeseries_only for only timeseries

### Possible scenario options ###
- Any scenario name. The script will search for input data for the specified this scenario and overwrite the default values.

In [1]:
# Options regarding output format and processing
output_file_format = 'csv'  # Change this to 'csv' or 'excel' depending on your needs
output_format = 'long' # Change to 'wide' or 'long' depending on your needs
processing_option = 'both' # Change to 'parameters_only' if you don't want the timeseries to created (saves time) or 'timeseries_only' for only timeseries or 'both' 
scenario_option = 'Europe_openENTRANCE_technoFriendly' # Specify your scenario here 

## Run this cell to execute the script ##
### You don't need to change anything in here ### 

In [2]:
from functions.function_import import master_function

# Call the function
master_function(output_file_format, output_format, processing_option, scenario_option)


Worksheets with specified scenario data:


Timeseries data overwritten by scenario data:


Everything worked successfully. You are great! Smile :)
